## TUGAS
Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca.

Prosedurnya adalah:

* Jalankan Model dan hitung loss dengan .
* Hitung update dan terapkan pada model dengan optimizer

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
# Print unique characters
for char in vocab:
    print(char, end=' ')


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [7]:
# VECTORIZE TEXT

example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
#  Membuat tf.keras.layers.StringLookup layer (Konvert Token jadi ID)

ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
# Mengembalikan ID ke Token

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
# Prediksi
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
# Membuat fungsi -> mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
# BATCHING TRAINING
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Membuat Model
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
# Uji Model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
sampled_indices

array([56, 58, 47, 12, 57,  7,  7, 34,  1,  9, 28, 20, 22, 52, 34, 13, 23,
       60, 17, 61, 36, 65, 56,  7, 36, 59, 43, 29, 22, 34, 52, 30, 10, 14,
       23, 12, 51, 28, 30, 35, 37, 45,  8, 51, 20, 26, 41, 36, 10, 42, 35,
       18, 50, 59, 15, 13, 14, 50, 53, 43, 23, 32,  8, 20, 51, 31,  1, 60,
       20, 18,  1, 12,  4, 21, 14, 39, 24,  7, 11, 23, 65, 42, 35, 34, 26,
       41, 29, 48, 61, 29,  3, 30, 17, 56, 46, 14, 42,  7,  4, 45])

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"When what's not meet, but what must be, was law,\nThen were they chosen: in a better hour,\nLet what i"

Next Char Predictions:
 b'qsh;r,,U\n.OGImU?JuDvWzq,WtdPIUmQ3AJ;lOQVXf-lGMbW3cVEktB?AkndJS-GlR\nuGE\n;$HAZK,:JzcVUMbPivP!QDqgAc,$f'


In [32]:
# Train Model
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.188841, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()

65.94631

In [35]:
model.compile(optimizer='adam', loss=loss)

In [36]:
# Checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
# Training
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 20s 58ms/step - loss: 2.7274
Epoch 2/10
172/172 [==============================] - 11s 52ms/step - loss: 1.9960
Epoch 3/10
172/172 [==============================] - 11s 53ms/step - loss: 1.7133
Epoch 4/10
172/172 [==============================] - 11s 54ms/step - loss: 1.5499
Epoch 5/10
172/172 [==============================] - 11s 56ms/step - loss: 1.4499
Epoch 6/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3811
Epoch 7/10
172/172 [==============================] - 12s 57ms/step - loss: 1.3291
Epoch 8/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2842
Epoch 9/10
172/172 [==============================] - 11s 55ms/step - loss: 1.2430
Epoch 10/10
172/172 [==============================] - 11s 56ms/step - loss: 1.2037


In [38]:
# Generate Teks
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Carest beg.
But what, you will sleep in deny.

BUCKINGHAM:
There's some promise, and the matter give striting fathing;
Which too riqudeapy, that my reputation be his
lived in my dam, more than you hope to play ourselves! Soft!--
A dight which I have not something agreed
In banishment, till they ather means,
Which is thine immortal toward, his heavy nature,
Nagure, one sickening, and so head,
For it be with such vill assised
and bear to tell me together, unds, uninhory
ThreelI thry you mean no opportes
A nacking on a sea, ere he, then behold
By yinged hands. What I come to-morrow,
Would swim in quine own toward the pressment-ball: hath
governantion that; I would be then, but yet my son was wretch
That makest hor, it should bear, talk with nothing;
And I can solely sort, to the people.

Second Murderer:
What but whose cour spirits him his kingdom sto mocking shame
By Courts, by namers of breath,
Ire you call drathless embraces, here, and a sisternoon,
No both us Clarence I seem: w

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nAll full your life,' so thrown the throne.\n\nDUKE OF YORK:\nAs if he doubt, so the shape or you and beat quiet,\nProofieg, sweet bastarry earth toge!\nOf what his death is short not honour in great?\nO, not so much, sir, and shall thou livest in ourselves?\n\nESCALUS:\nMore!' belike my son, the park of Buckingham;\nFor what are you mine,\nThere shall pack not of him.\n\nCATESBY:\nMy lord, by nothing speaks botlant,\nFiend with blazes, winteful last issues!\nSitifulance of quick, window. What ever you misthose soft worse?\n\nELBOW:\nFaith, no more: something mad, some stopp'd, that's paesenge.\nO, do thy flight--must needs to thum;\nHis coudsels, his youth as you, my noble son,\nThis sweet belay-like would say, will find my pardon.\n\nHASTINGS:\nYou are by this provost; fellow only.\n\nCLIFFORD:\nIf I have asparent to the threater,\nEre I must, she will not, I hear,\nThe sun hath strange queen, and so us down,\nWith all three joints on my sword,\nTrainous youth to

In [42]:
# Ekspor Model Generator
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [43]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
O, if ere not at your ignorant father, good fello,
With all storns that come, these thought you wit


In [44]:
# JAWABAN
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [45]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [47]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 56ms/step - loss: 2.7206


In [48]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print()
  print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
  print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
  print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1777
Epoch 1 Batch 50 Loss 2.1126
Epoch 1 Batch 100 Loss 1.9446
Epoch 1 Batch 150 Loss 1.8546

Epoch 1 Loss: 1.9902
Time taken for 1 epoch 12.93 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8387
Epoch 2 Batch 50 Loss 1.7133
Epoch 2 Batch 100 Loss 1.6759
Epoch 2 Batch 150 Loss 1.6603

Epoch 2 Loss: 1.7091
Time taken for 1 epoch 20.47 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6016
Epoch 3 Batch 50 Loss 1.5818
Epoch 3 Batch 100 Loss 1.5705
Epoch 3 Batch 150 Loss 1.5184

Epoch 3 Loss: 1.5483
Time taken for 1 epoch 11.65 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4506
Epoch 4 Batch 50 Loss 1.4900
Epoch 4 Batch 100 Loss 1.4687
Epoch 4 Batch 150 Loss 1.4648

Epoch 4 Loss: 1.4494
Time taken for 1 epoch 12.05 sec
_____________________________________________________________________

### JAWABAN SOAL 2
Perbedaan mencolok antara kode tugas dan praktikum 2 terletak pada pendekatan pelatihan. Praktikum 2 memilih jalur yang lebih simpel dengan model.fit, sedangkan tugas menunjukkan pendekatan yang lebih kompleks. Dalam pendekatan ini, melibatkan definisi train_step dalam model turunan untuk mengatur pelatihan pada tingkat batch. Langkah-langkah eksplisit seperti perhitungan loss, gradien, dan pembaruan bobot model dengan apply_gradients digunakan, disertai dengan penggunaan objek tf.metrics.Mean untuk mengukur rata-rata loss selama pelatihan. Keseluruhan, pendekatan ini memberikan tingkat kontrol dan fleksibilitas yang lebih tinggi dalam mengelola pelatihan model.